In [ ]:
import pandas as pd
import numpy as np

In [ ]:
class Node:
    def __init__(self, feature_to_split=None, threshold=None, left=None, right=None, data=None):
        self.feature_to_split = feature_to_split
        self.threshold = threshold
        self.left = left
        self.right = right
        self.data = data

In [ ]:
class Custom_xgb:
    def __init__(self, max_depth=3, min_n_samples=2):
        self.max_depth = max_depth
        self.min_n_samples = min_n_samples
        self.root = None

    def gain_variance(self, y: pd.DataFrame, y_left: pd.DataFrame, y_right: pd.DataFrame):
        y_var = np.var(y)
        y_left_var = np.var(y_left)
        y_right_var = np.var(y_right)

        weighted_var = (len(y_left)*y_left_var + len(y_right)*y_right_var)/len(y)

        return (y_var - weighted_var)
        



    def find_best_split(self, X: pd.DataFrame, y: pd.DataFrame):        

        # store and update the bests when a better gain is found
        best_feature = None
        best_gain = -1000
        best_threshold = None
        best_mask = None

        # take a feature, then go through all values of that feature, taking them as the new threshold to check gain
        for feature in X.columns: 
            thresholds = X[feature].unique()
            for threshold in thresholds:
                # divide into y_left and y_right
                mask = X[feature] <= threshold
                y_left = y[mask]
                y_right = y[~mask]

                if len(y_left)==0 or len(y_right)==0:
                    continue

                # check gain for the current threshold and feature, and update if a better gain is found
                current_gain = self.gain_variance(y, y_left, y_right)
                if current_gain > best_gain:
                    best_gain = current_gain
                    best_feature = feature
                    best_threshold = threshold
                    best_mask = mask
            

        if best_feature is None:
            return None, None, None, None, None, None
        

        # construct the x_left, y_left, x_right, y_right according to the best feature and threshold
        X_left = X[best_mask]
        X_right = X[~best_mask]
        
        y_left = y[best_mask]
        y_right = y[~best_mask]



        return (best_feature, best_threshold, X_left, y_left, X_right, y_right)
    

    def build_tree(self, X: pd.DataFrame, y: pd.DataFrame, depth=0):
        n_samples, _ = X.shape

        # if we satisfy the conditions, we return a leaf node
        if depth >= self.max_depth or n_samples < self.min_n_samples:
            leaf_mean = np.mean(y)
            return Node(data=leaf_mean)
        
        
        
        best_feature, best_threshold, X_left, y_left, X_right, y_right = self.find_best_split(X, y) 

        # return a leaf node if no gain was found
        if best_feature is None:
            leaf_mean = np.mean(y)
            return Node(data=leaf_mean)


        left_child = self.build_tree(X_left, y_left, depth+1)
        right_child = self.build_tree(X_right, y_right, depth+1)

        return Node(feature_to_split=best_feature, threshold=best_threshold, left=left_child, right=right_child)

        

    def fit(self, X, y):
        self.root = self.build_tree(X, y)


    def predict(self, X):
        predictions = []

        for row_i in range(len(X)):
            row = X.iloc[row_i]

            current_node = self.root

            while current_node.data is None:
                feature = current_node.feature_to_split
                threshold = current_node.threshold
                
                
                if row[feature] <= threshold:
                    current_node = current_node.left
                else:
                    current_node = current_node.right

            predictions.append(current_node.data)

        return np.array(predictions)




        # gains = dict()
        # max_gain = 0

        # # for categorical data. OHE only
        # for feature in X.columns:
        #     for row in range(len(X)):
        #         if X[row][feature]:
        #             pd.concat([y_left, y[row]])
        #         else:
        #             pd.concat([y_right, y[row]])

        #     current_gain = self.gain_variance(y, y_left, y_right)

        #     if current_gain > max_gain:
        #         max_gain = current_gain
            
        #     gains[str(current_gain)] = feature

        # feature_to_split = gains[str(max_gain)]


        # X_left = pd.DataFrame()
        # X_left.columns = X.columns
        # X_right = pd.DataFrame()
        # X_right.columns = X.columns


        # for row in range(len(X)):
        #     if X[row][feature_to_split]:
        #         pd.concat([X_left, X[row]])
        #     else:
        #         pd.concat([X_right, X[row]])

        
        # for continuous (will later delete the above categorical)


        # ---------------------------------------------

    # def entropy(self, X: pd.DataFrame, feature: str):
    #     successes = np.sum(X[feature])

    #     if successes == 0 or successes == len(X):
    #         return 1
        
    #     else:
    #         p1 = np.sum(X[feature])/len(X)
    #         p2 = 1-p1
    #         return -p1*np.log2(p1) - p2*np.log2(p2)



    # def gain_entropy(self, X: pd.DataFrame, X_left: pd.DataFrame, X_right: pd.DataFrame, feature: str):
    #     x_entropy = self.entropy(X, feature)
    #     x_left_entropy = self.entropy(X_left, feature)
    #     x_right_entropy = self.entropy(X_right, feature)

    #     weighted_entropy = len(X_left)*x_left_entropy + len(X_right)*x_right_entropy

    #     return (x_entropy - weighted_entropy)
        